## Problem 1 and 2 combined

In [58]:
import os
import openpyxl
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import string
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sklearn.metrics.pairwise
from sklearn.decomposition import PCA
import seaborn as sns

'''
os.getcwd()
keywords = []
keywords2 = []
file = 'keywords.xlsx'
data = pd.ExcelFile(file)
df5 = data.parse('Sheet1')
ps = openpyxl.load_workbook('keywords.xlsx')
sheet = ps['Sheet1']
for row in range(2, sheet.max_row + 1):
    keywords.append(sheet['A' + str(row)].value)
for keyword in keywords:
    keywords2.append(keyword.replace(" ", "+"))
keywords = keywords2[:]
'''

keywords2 = ['cornavirus','uk','world','business','politics','technology','science_and_environment','health','education']
keywords = keywords2[:]

In [59]:
print(keywords2)

['cornavirus', 'uk', 'world', 'business', 'politics', 'technology', 'science_and_environment', 'health', 'education']


In [40]:
urls = []
for keyword in keywords2:
    keywordurls = []
    for j in range(1,1000):
        theurl = "https://www.bbc.co.uk/{}".format(keyword)
        thepage = urllib.request.urlopen(theurl)
        soup = BeautifulSoup(thepage)
        project_href = [i['href'] for i in soup.find_all('a', href=True)]
        if len([i for i, s in enumerate(project_href) if '/search?q=' in s])==0:
            if j != 1:
                break
            else:
                keywordurls.append(project_href[project_href.index('#more-menu-button')+1:[i for i, s in enumerate(project_href) if s =='https://www.bbc.co.uk'][3]-4])
        else:
            keywordurls.append(project_href[project_href.index('#more-menu-button')+1:[i for i, s in enumerate(project_href) if '/search?q=' in s][0]])
        print(j)
    urls.append(keywordurls)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [49]:
print(len(urls))

10


In [54]:

urls2 = [[] for i in range(len(keywords))]
for i in range(len(urls)):
    for j in range(len(urls[i])):
        for k in range(len(urls[i][j])):
            urls2[i].append(urls[i][j][k])

print(urls2)
#urls3 = urls[:]

[['https://www.bbc.co.uk/programmes/m000rh4p', 'https://www.bbc.co.uk/news/uk-england-beds-bucks-herts-62138005', 'https://www.bbc.co.uk/news/uk-england-south-yorkshire-62119826', 'https://www.bbc.co.uk/news/uk-scotland-tayside-central-62062683', 'https://www.bbc.co.uk/news/uk-england-london-62050348', 'https://www.bbc.co.uk/news/uk-northern-ireland-30175412', 'http://www.bbc.co.uk/news/world-europe-41251684', 'https://www.bbc.co.uk/news/uk-northern-ireland-24171380', 'http://news.bbc.co.uk/1/hi/uk_politics/7736794.stm', 'https://www.bbc.co.uk/news/uk-england-sussex-11969289', 'https://www.bbc.co.uk/news/uk-england-derbyshire-62001406', 'https://www.bbc.co.uk/news/newsbeat-61992772', 'https://www.bbc.co.uk/news/uk-england-leicestershire-61958079', 'https://www.bbc.co.uk/news/uk-england-shropshire-61899885', 'https://www.bbc.co.uk/sport/football/61851617', 'https://www.bbc.co.uk/news/uk-wales-61838960', 'https://www.bbc.com/news/articles/cv2xygjrld3o', 'https://www.bbc.co.uk/programmes/

[10, 9, 7, 5, 3, 2, 1, 0]


In [55]:
def list_delete_indices(lst, indices):
    for idx in sorted(indices, reverse=True):
        if idx < len(lst):
            lst.pop(idx)

urls4 = [[] for i in range(len(keywords))]
for i in range(len(urls2)):
    if len(urls2[i]) > 100:
        indices_list = [i for i, s in enumerate(urls2[i]) if 'https://www.bbc.co.uk/programmes' in s or 'https://www.bbc.co.uk/sport' in s or 'http://news.bbc.co.uk/sport1' in s or 'http://www.bbc.co.uk/sport' in s or 'http://www.bbc.co.uk/programmes' in s or 'http://news.bbc.co.uk/1/hi/programmes' in s or 'http://www.bbc.co.uk/guides' in s or 'https://www.bbc.co.uk/newsround' in s or 'https://www.bbc.co.uk/news/av' in s]
        list_delete_indices(urls2[i], indices_list)
        tech = [i for i, s in enumerate(urls2[i]) if 'https://www.bbc.co.uk/news/technology' in s]
        for j in tech:
            urls4[i].append(urls2[i][j])
        list_delete_indices(urls2[i], tech)
        for k in range(100-len(urls4[i])):
            urls4[i].append(urls2[i][k])
    else:
        for j in range(len(urls2[i])):
            urls4[i].append(urls2[i][j])
        
print(urls4)

IndexError: list index out of range

In [312]:
for i in range(len(urls4)):
    for j in range(len(urls4[i])):
        urllib.request.urlretrieve(urls4[i][j],'C:\\Users\\Ian\\Documents\\Data_C&C_Practicals\\htmlcontent'+str(i)+str(j)+'.txt')

        file = open('htmlcontent'+str(i)+str(j)+'.txt', "r", errors="ignore")
        contents = file.read()
        soup = BeautifulSoup(contents, 'html.parser')

        f = open('C:\\Users\\Ian\\Documents\\Data_C&C_Practicals\\articlecontent'+str(i)+str(j)+'.txt', "w", errors="ignore")
        
        try:
            f.writelines(soup.find(id='main-heading').get_text())
            f.write(' ')
            
            for data in soup.find_all('div', class_='ssrcss-3z08n3-RichTextContainer e5tfeyi2'):
                f.writelines(data.get_text())
                f.write(' ')

            for data in soup.find_all('div', class_='ssrcss-mysbf6-ComponentWrapper-CrossheadComponentWrapper e1xue1i84'):
                f.writelines(data.get_text())
                f.write(' ')

            for data in soup.find_all('a', class_='ssrcss-1yno9a1-StyledLink ed0g1kj1'):
                f.writelines(data.get_text())
                f.write(' ')
                
        except AttributeError:
            try:
                f.writelines(soup.find_all('div', class_='mxb')[0].get_text())
                f.write(' ')

                f.writelines(soup.find_all('td', class_ = 'storybody')[0].get_text())
                f.write(' ')
            except IndexError:
                for data in soup.find_all('p'):
                    f.writelines(data.get_text())

        f.close()


## Problem 3

In [156]:
stemmed_list = []
for i in range(len(urls4)):
    for j in range(len(urls4[i])):
        filename = theurl = "articlecontent{}{}.txt".format(i,j)
        file = open(filename, 'rt')
        text = file.read()
        file.close()
        # split into words by white space
        words = text.replace('.', ' ')
        words = re.sub('[^a-zA-Z0-9\n\.]', ' ', words)
        for t in string.punctuation:
            words = words.replace(str(t), " ")
        words = words.split()
        words = [c for c in words if not c.isdigit()]
        words = [''.join([i for i in s if not i.isdigit()]) for s in words]
        # convert to lower case
        words = [word.lower() for word in words]
        stop_words = set(stopwords.words('english'))
        words2 = [w for w in words if not w in stop_words]
        #print(len(words2))
        #print(words2)
        porter = PorterStemmer()
        stemmed = [porter.stem(word) for word in words2]
        #print(stemmed)
        stemmed = ' '.join(stemmed)
        print(stemmed)
        stemmed_list.append(stemmed)

coronaviru hacker target covid vaccin suppli cold chain intern vaccin suppli chain target cyber espionag accord ibm compani say track campaign aim deliveri cold chain use keep vaccin right temperatur transport attack ident unclear ibm said sophist method indic nation state follow warn govern includ uk countri target aspect vaccin research ibm say believ campaign start septemb say phish email sent across six countri target organis link cold chain equip optimis platform cceop gavi intern vaccin allianc gavi partner includ world health organ unicef world bank bill melinda gate foundat help distribut vaccin around world poorest region sometim requir cold chain exampl pfizer biontech vaccin specif target campaign need kept temperatur c move attack imperson busi execut legitim chines compani involv cceop suppli cold chain make like target would engag email sent phish email organis provid transport contain malici code ask peopl log credenti could allow understand infrastructur govern intend u

bristol theatr archiv grant save risk record univers bristol theatr archiv receiv govern grant stop valuabl record lost forev theatr collect one world lead archiv british theatr get nation archiv help identifi item uk risk ensur preserv perform art sector pressur due covid import record said jeopardi professor karla pollman dean faculti art univers bristol said grate nation archiv award grant respons urgent situat money help safeguard digit well analogu materi photograph design actor writer deem immedi danger ad project potenti make materi contribut histori theatr live art britain might otherwis disappear collect document life stage hope safeguard home vulner collect commun grant allow archivist target collect threat across spectrum british perform practic compani individu backstag profession safe store record total archiv across england scotland wale northern ireland receiv fund govern grant offset practic cost keep archiv storag conserv transport expertis jeff jame chief execut keepe

russian citi target mysteri bomb threat four day run town citi across russia hit wave bomb alert dozen school shop centr railway station public build evacu ten thousand peopl affect far alert prove hoax public urg remain calm sourc threat unclear one offici suggest origin outsid russia reason assum organis abroad offici chelyabinsk told interfax news agenc wednesday bomb scare spread moscow prompt evacu luxuri gum shop centr near kremlin capit alon peopl caught alert report said pro kremlin newspap point major hack attack possibl ukrain state tv said compani organis target nation massiv cyber attack hit countrieswher ransomwar hit hardest russian hacker target macron campaign report suggest threat made via scrambl internet connect difficult sourc one offici quot say evacu part anti terrorist drill confirm hoax call began sunday extent alert becam clear tuesday chelyabinsk southern ural mountain peopl clear build tuesdayin perm north west peopl leav school public buildingsin southern ci

bristol artist get racist abus joy project black artist target threat violenc racist abus follow campaign social chang women took part rise art agenc whosefutur campaign billboard bristol post art photographi poetri statement agenc said minor peopl target six artist well agenc avon somerset polic confirm investig report made social media sever individu campaign launch mid juli respons worldwid black live matter movement founder kamina walton said major abus aim one young black femal artist agenc name order protect mixtur extrem angri upset traumatis knew potenti backlash idea would point peopl live threaten ad ms walton agenc also said receiv public support aim give voic peopl colour felt ostracis exclud art sector campaign ask question citi also say look talent doorstep see joy campaign artist self esteem roof felt like real celebr said ms walton agenc said continu work encourag open honesti bristol citi council said statement art interpret therefor alway draw differ opinion peopl may

petrol bomb thrown shrewsburi hous arson petrol bomb believ thrown hous arson attack shropshir fire start garden properti mari webb road shrewsburi gmt one injur damag caus properti west mercia polic said insp paul mathison forc said believ target attack wider threat public shrewsburi
polic target hooligan threat strathclyd forc said take threat troubl extrem serious extra offic draft secur step port airport train station ahead vital match major polic oper also involv german offic travel glasgow identifi troublemak polic receiv intellig report suggest known hooligan gather glasgow across scotland elsewher uk offic strathclyd also receiv inform german counterpart warn rival troublemak plan travel scotland said polic spokeswoman potenti troubl strathclyd polic mount massiv offens counter threat hooligan dedic intellig oper set deal potenti troubl saturday chief superintend john malcolm command day match said forc work close sever scottish forc nation crimin intellig servic german polic k

...

intel zombieload bug fix slow data centr comput intel confirm new problem discov processor chip mean comput owner face perform slowdown compani said data centr like worst affect fix requir ad impact pc owner minim call zombieload vulner follow disclosur earlier spectr meltdown foreshadow bug last year latest flaw could theoret allow attack spi task handl intel core xeon brand central process unit cpu releas sinc amazon web servic microsoft azur googl cloud among major cloud comput platform power data centr technolog taken step avoid client notic impact put risk tech giant may need invest extra comput server softwar patch involv take major toll perform zombieload discov research graz univers technolog austria ku leuven univers belgium said could allow hacker steal sensit data provid mean unscrambl encrypt file could affect user level secret browser histori websit content user key password system level secret disk encrypt key explain ad current unclear whether attack would leav trace any

messag encrypt problem rudd major technolog compani must step fight extrem face new law home secretari told bbc amber rudd said technolog compani enough beat enemi internet encrypt tool use messag app becom problem ad ms rudd meet repres googl facebook twitter microsoft other counter terror forum san francisco tuesday summit first gather global internet forum counter terror organis set major compani wake recent terror attack joint statement compani take part said co oper substanti disrupt terrorist abil use internet further caus also respect human right whatsapp must place terrorist hide euro mp back end end encrypt citizensaustralian pm seek access encrypt messag ms rudd expect tell compani extremist allow upload content realli tri achiev told bbc technolog compani say us use artifici intellig begin make progress way privaci right group electron frontier foundat express concern possibl heavi hand block content said move would signific impact free speech onlin encrypt common featur bus

phone encrypt polic mug suspect get data detect develop new tactic beat crimin use mobil phone encrypt legal mug tactic emerg scotland yard cybercrim unit smash fake credit card fraud racket offic realis crucial evid investig conceal suspect iphon would unobtain devic lock covert team seiz street suspect call beat secur set street seizur phone dreamt detect oper falcon specialist metropolitan polic team run investig major fraud relat crime organis onlin gabriel yew investig suspect manufactur fake card gang use across europ buy luxuri good detect suspect use iphon exclus commun member network knew arrest could refus unlock would never see incrimin evid consid whether could legal forc suspect finger thumb devic fingerprint reader unlock found power howev conclud could stage law street robberi use similar snatch techniqu thief june team set precis undercov surveil offic trail yew wait unlock phone make call therebi disabl encrypt one offic rush seiz phone yew hand would happen crimin mug

govern lose control encrypt clash appl fbi whether compani provid access encrypt data lock iphon use one san bernardino attack highlight debat privaci data secur rage decad cryptographi control state deploy militari diplomat end cryptograph whitfield diffi devis system took encrypt key away state mark start call crypto war whitfield diffi three expert spoke bbc world servic inquiri programm tension heart spat appl fbi cryptograph whitfield diffi devis public key cryptographi revolutionis encrypt basic techniqu use public key cryptographi come around western world known middl east basic arithmet ordinari integ arithmet someth like clock arithmet clock wait three hour get clock tabl lookup th element tabl th element troubl well without kind mechan comput human enough calcul produc secur system without make mani mistak could support cryptographi outsid fairli integr organis look us depart defens larg centralis everybodi know chain command trust entiti run nation secur agenc nsa manag key 

pari attack silicon valley crosshair encrypt grief pari attack soon make way demand action well increas militari activ controversi suggest close door refuge next battl sure someth done arena aim squar angrili silicon valley tech compani alreadi pressur make easier govern access privat commun app servic call intensifi greatli sinc attack pari creat product allow evil monster commun way behead children strike innoc whether game stadium small restaur pari take airlin big problem diann feinstein chair senat intellig committe told msnbc need hi tech help secur internet even court order get say big problem problem encrypt without encrypt thing onlin would insecur email shop bank reli principl encrypt data use complex mathemat nigh imposs crack use commun app whatsapp appl imessag wechat messag encrypt default mean even compani want hand messag law enforc bad say lot technolog capabl avail right make except difficult technic well legal intellig secur servic insight need uncov said cia directo

In [210]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(stemmed_list)

In [239]:
pca = PCA(n_components=700)
pca.fit(X.todense())
PCA_X = pca.fit_transform(X.todense())
print(sum(pca.explained_variance_ratio_))

0.9924047272114245


In [201]:
df = pd.DataFrame(np.array(sklearn.metrics.pairwise.cosine_similarity(PCA_X)),index = [str(i) for i in range(PCA_X.shape[0])], columns = [str(i) for i in range(PCA_X.shape[0])])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,811,812,813,814,815,816,817,818,819,820
0,1.000000,0.137701,0.002759,0.001245,-0.020501,-0.013394,0.013526,-0.018288,0.010456,-0.004761,...,-0.020777,-0.012155,-0.004194,-0.014817,-0.024133,-0.024433,-0.024506,-0.014922,-0.015638,-0.014505
1,0.137701,1.000000,-0.031951,-0.020477,-0.034274,-0.027661,-0.015816,0.005078,0.018637,-0.034738,...,0.006369,0.014761,0.027159,-0.041409,-0.028582,-0.004716,-0.045567,-0.014591,0.017615,-0.023661
2,0.002759,-0.031951,1.000000,0.022309,0.018582,0.006737,0.008850,0.009390,0.000188,0.009777,...,-0.009913,-0.003618,-0.004994,0.015575,0.023732,-0.009684,-0.012767,0.009425,0.006127,-0.020448
3,0.001245,-0.020477,0.022309,1.000000,0.279914,0.018474,-0.002926,0.057480,0.041719,0.025942,...,-0.022883,-0.047929,-0.036267,-0.015230,-0.012651,-0.031526,-0.017767,-0.017891,-0.031473,-0.021668
4,-0.020501,-0.034274,0.018582,0.279914,1.000000,0.025259,0.000262,0.045901,0.045497,0.048092,...,-0.020024,-0.047784,-0.032698,-0.003667,-0.038141,-0.009049,-0.022310,-0.005106,-0.026397,-0.035688


In [207]:
averages = [[0 for j in range(len(keywords))] for i in range(len(keywords))]
n = 30
c = 0
for l in range(len(keywords)):
    d = 0
    for k in range(len(keywords)):
        if k < l:
            s = []
            for i in range(len(urls4[l])):
                for j in range(len(urls4[k])):
                    #print(df.iloc[i+c,j+d])
                    s.append(df.iloc[i+c,j+d])
            averages[l][k] = sum(sorted(s, reverse=True)[:n])/(n)
            d += len(urls4[k])
        elif k == l:
            averages[l][k] = 1
    c += len(urls4[l])

In [208]:
my_array = np.array(averages)
keywords3 = []
for keyword in keywords:
    keywords3.append(keyword.replace("+", " "))
keywords = keywords3[:]
df2 = pd.DataFrame(my_array, columns = keywords, index = keywords)
df2 = 1 - df2
print(df2)

df2.to_excel('distance.xlsx')





                            targeted threat  Advanced Persistent Threat  \
targeted threat                    0.000000                    1.000000   
Advanced Persistent Threat         0.912691                    0.000000   
phishing                           0.759818                    0.859492   
DoS attack                         0.743157                    0.928162   
malware                            0.840452                    0.824786   
computer virus                     0.775944                    0.894813   
spyware                            0.828205                    0.853685   
malicious bot                      0.952241                    0.969338   
ransomware                         0.787756                    0.838828   
encryption                         0.806779                    0.871305   

                            phishing  DoS attack   malware  computer virus  \
targeted threat             1.000000    1.000000  1.000000        1.000000   
Advanced Persisten

## Problem 4

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
X2 = vectorizer.fit_transform(stemmed_list)

In [ ]:
keywords4 = []
for keyword in keywords:
    keywords4.append(keyword.replace("+", " "))
keywords = keywords4[:]
keywords = [' '.join([porter.stem(word) for word in sentence.split(" ")]) for sentence in keywords]
print(keywords)
similarity = [[0 for j in range(len(keywords))] for i in range(len(keywords))]
for i in range(len(keywords)):
    for j in range(len(keywords)):
        if i > j:
            iindex=vectorizer.get_feature_names().index(keywords[i])
            jindex=vectorizer.get_feature_names().index(keywords[j])
            ivector=X2[:,[iindex]].toarray()
            jvector=X2[:,[jindex]].toarray()
            similarity[i][j] = sklearn.metrics.pairwise.cosine_similarity(np.transpose(ivector),np.transpose(jvector))[0][0]
        elif i == j:
            similarity[i][j] = 1
print(similarity)

In [ ]:
keywords4 = []
for keyword in keywords:
    keywords4.append(keyword.replace("+", " "))
keywords = keywords4[:]
keywords = [' '.join([porter.stem(word) for word in sentence.split(" ")]) for sentence in keywords]
#print(keywords)
similarity = []
for i in range(len(keywords)):
    iindex=vectorizer.get_feature_names().index(keywords[i])
    similarity.append(X2[:,[iindex]].toarray())

In [ ]:
similarity2 = [[] for i in range(len(keywords))]
for i in range(len(keywords)):
    for j in range(len(similarity[i])):
        similarity2[i].append(float(similarity[i][j]))
print(similarity2)

In [ ]:
malwarefig = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(len(similarity2[0]))],[similarity2[4][i] for i in range(len(similarity2[0]))]])),index = [str(i) for i in range(len(similarity2[0]))], columns = ['Document', 'tf-idf score'])
sns_plot = sns.lineplot(x='Document', y='tf-idf score',data=malwarefig)
fig = sns_plot.get_figure()
fig.savefig('malwarefig.png')

In [ ]:
phishingfig = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(len(similarity2[0]))],[similarity2[2][i] for i in range(len(similarity2[0]))]])),index = [str(i) for i in range(len(similarity2[0]))], columns = ['Document', 'tf-idf score'])
sns_plot = sns.lineplot(x='Document', y='tf-idf score',data=phishingfig)
fig = sns_plot.get_figure()
fig.savefig('phishingfig.png')

In [ ]:
cumulative_variances = []
variances = []

l = 700

pca = PCA(n_components=l)
pca.fit(X.todense())

for i in range(l):
    variances.append(pca.explained_variance_ratio_[i])
    cumulative_variances.append(sum(pca.explained_variance_ratio_[:i]))

In [ ]:
pca1 = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(l)],variances])),index = [str(i) for i in range(l)], columns = ['Principal component', 'Proportion of variance explained'])
sns_plot = sns.lineplot(x='Principal component', y='Proportion of variance explained',data=pca1)
fig = sns_plot.get_figure()
fig.savefig('pca1.png') 

In [ ]:
pca2 = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(l)],cumulative_variances])),index = [str(i) for i in range(l)], columns = ['Principal component', 'Cumulative proportion of variance explained'])
sns_plot = sns.lineplot(x='Principal component', y='Cumulative proportion of variance explained',data=pca2)
fig = sns_plot.get_figure()
fig.savefig('pca2.png') 

In [ ]:
pca3 = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(l)],[PCA_X[0][i] for i in range(l)]])),index = [str(i) for i in range(l)], columns = ['Principal component', 'tf-idf score'])
sns_plot = sns.lineplot(x='Principal component', y='tf-idf score',data=pca3)
fig = sns_plot.get_figure()
fig.savefig('pca3.png') 

In [ ]:
pca4 = pd.DataFrame(np.transpose(np.array([[i+1 for i in range(l)],[PCA_X[617][i] for i in range(l)]])),index = [str(i) for i in range(l)], columns = ['Principal component', 'tf-idf score'])
sns_plot = sns.lineplot(x='Principal component', y='tf-idf score',data=pca4)
fig = sns_plot.get_figure()
fig.savefig('pca4.png') 

In [ ]:
ax = sns.heatmap(df2)